**Simulated Annealing** is a stochastic global search optimization algorithm. This means that it makes use of randomness as part of the search process. This makes the algorithm appropriate for nonlinear objective functions where other local search algorithms do not operate well.

 The main advantage of SA is its ability to escape from local minima and converge to a global minimum. Like the stochastic hill climbing local search algorithm, it modifies a single solution and searches the relatively local area of the search space until the local optima is located. Unlike the hill climbing algorithm, it may accept worse solutions as the current working solution.

The likelihood of accepting worse solutions starts high at the beginning of the search and decreases with the progress of the search, giving the algorithm the opportunity to first locate the region for the global optima, escaping local optima, then hill climb to the optima itself.

In [3]:
import random as rn
import re
import numpy as np
import math
from numpy import random

The algorithm generated paterns of the requests input. Based on the pattern, the sum of elements form i = 0 to i = length_os_solution is calculated and whenever the sum exceeds the limit, the number of rolls is incremented.

The number of neighbors is determined based on the length of the stock. When selecting the next solution, the cost of each neighbor is calculated, and, as one would expect, the one with the most favorable cost is chosen for the subsequent iteration. However, this algorithm differs from hill climbing in that there is a probability factor that influences the selection of which neighbor should be chosen for the next round. When the temperature is high, there is a greater likelihood that a neighbor with a less favorable cost will be selected. At first glance, this might appear to be a disadvantage, but this variation allows the algorithm to escape local optima and explore other areas of the solution space.


The SimulatedAnnealing class consists of 5 methods:
1.   read_file(file_name): the input file is stored in variable *inp* then the string is splitted and only the digits remain. We turn the characters to integers in order to easily calculate cost in the next fucntion.
2.   cost_function(sol): For the solution *sol*, the sum of elements form i = 0 to i = length_os_solution is calculated. during calculation if the *curr_sum* exceeds the stock_length, number of rolls is incremented. The output is a percentage of optimal solution. If the cost is 100% it means the desired solution is achieved, if cost < 100% the number of rolls is higher than the desired answer and if cost > 100% we have reached a optimal solution.
3. swap(i, j, ls): a utility function to swap two elements in list *ls*
4. generate_solution(): In the first step, a solution need to be generated. this solution is the same *requests* list, except that the members are shuffled.
5. generate_neighbours(sol): For each solution number of *n_neighbours* are generated by swaping 2 elements of the solution. This way the neighbours are slightly different from solution and have a new pattern.



In [34]:
class SimulatedAnnealing():
    def __init__(self, file_name):
        self.stock_len, self.target, self.requests = self.read_file(file_name)
        self.n_req = len(self.requests)
        self.n_neighbours = math.ceil(self.stock_len * 1/7)
        self.alpha = 0.991

    def read_file(self, file_name):
        f = open(file_name)
        inp = f.read()
        f.close()
        # Only digits in form of characters are stored
        req = re.split('\D+', inp)
        req = req[1:-1]
        # Cast char to int
        req = list(map(int, req))
        stock_len = req[0]
        ans = req[-1]
        req = req[1:-1]
        return stock_len, ans, req

    def cost_function(self, sol):
        n_rolls = 0
        curr_sum = 0
        # Sum based on solution's pattern
        for i in sol:
            curr_sum += i
            if curr_sum > self.stock_len:
                n_rolls += 1
                curr_sum = i
        return self.target/n_rolls*100

    # Utility function
    def swap(self, i, j, ls):
        tmp = ls[i]
        ls[i] = ls[j]
        ls[j] = tmp
        return ls

    def generate_solution(self):
        sol = self.requests.copy()
        random.shuffle(sol)
        return sol

    def generate_neighbours(self, sol):
        neighbours = []
        temp = []
        for _ in range(self.n_neighbours):
            # 2 random members of solution are swapped leading to a new pattern
            neighbours.append(self.swap(random.randint(0, len(sol)), random.randint(0, len(sol)), sol.copy()))
        return neighbours

In [156]:
def optimizer(file_name):
    # Reads the input
    SA = SimulatedAnnealing(file_name)
    # Initialize solution
    curr_sol = SA.generate_solution()
    curr_cost = SA.cost_function(curr_sol)

    it = 0
    max_it = 2000
    temp = 1

    while it < max_it:
        it += 1
        for i in SA.generate_neighbours(curr_sol):
            neighbour_cost = SA.cost_function(i)
            # selects the neighbout with highest profit
            if neighbour_cost >= curr_cost:
                curr_sol = i
                curr_cost = SA.cost_function(i)
            # If the temperature is high, it is more likely that the next three lines will be executed.
            elif np.random.choice([0, 1], 1, p = [1 - temp, temp]):
                curr_sol = i
                curr_cost = SA.cost_function(i)
        if it % 100 == 0:
            print(f"Iteration {it} - Temperature {temp} - Best Solution {curr_cost} - Rolls No. {SA.target/curr_cost*100}")
        # alpha is temperature cooling rate, the temp decreases as the iterations passes
        temp *= SA.alpha

Below are the results of the algorithm when applied to the four input files. It should be noted that this algorithm performs as well as, or better than, the hill climbing algorithm within the same number of iterations. This algorithm is designed with the flexibility to escape local optima, allowing it to explore more of the solution space.

### The **best solution** after 2000 iteration is: 50 rolls
The solutions don’t always show a decreasing trend. For example, in iteration 700, the number of rolls is 51, but in iteration 800, it increases to 52. As the temperature decreases, these kinds of fluctuations become less frequent.

In [157]:
optimizer("input1.stock")

Iteration 100 - Temperature 0.40859382039498143 - Best Solution 83.60655737704919 - Rolls No. 61.0
Iteration 200 - Temperature 0.16544636987438172 - Best Solution 94.44444444444444 - Rolls No. 54.0
Iteration 300 - Temperature 0.06699196105841773 - Best Solution 94.44444444444444 - Rolls No. 54.0
Iteration 400 - Temperature 0.027126148792869235 - Best Solution 96.22641509433963 - Rolls No. 52.99999999999999
Iteration 500 - Temperature 0.010983824576970218 - Best Solution 98.07692307692307 - Rolls No. 52.0
Iteration 600 - Temperature 0.00444753154083448 - Best Solution 100.0 - Rolls No. 51.0
Iteration 700 - Temperature 0.0018008787984643694 - Best Solution 100.0 - Rolls No. 51.0
Iteration 800 - Temperature 0.0007292054967978848 - Best Solution 98.07692307692307 - Rolls No. 52.0
Iteration 900 - Temperature 0.00029526732005156104 - Best Solution 98.07692307692307 - Rolls No. 52.0
Iteration 1000 - Temperature 0.00011955860271661615 - Best Solution 100.0 - Rolls No. 51.0
Iteration 1100 - Tem

### The **best solution** after 2000 iteration is: 77 rolls
As a result of these fluctuations, the solution can cover a wider range of outcomes, leading to results that are better than those from the hill climbing algorithm. However, this output may also arise from the inherent randomness of the algorithm.

In [165]:
optimizer("input2.stock")

Iteration 100 - Temperature 0.40859382039498143 - Best Solution 83.9080459770115 - Rolls No. 87.0
Iteration 200 - Temperature 0.16544636987438172 - Best Solution 86.90476190476191 - Rolls No. 84.0
Iteration 300 - Temperature 0.06699196105841773 - Best Solution 86.90476190476191 - Rolls No. 84.0
Iteration 400 - Temperature 0.027126148792869235 - Best Solution 87.95180722891565 - Rolls No. 83.0
Iteration 500 - Temperature 0.010983824576970218 - Best Solution 89.02439024390245 - Rolls No. 82.0
Iteration 600 - Temperature 0.00444753154083448 - Best Solution 90.12345679012346 - Rolls No. 81.0
Iteration 700 - Temperature 0.0018008787984643694 - Best Solution 92.40506329113924 - Rolls No. 79.0
Iteration 800 - Temperature 0.0007292054967978848 - Best Solution 93.58974358974359 - Rolls No. 78.0
Iteration 900 - Temperature 0.00029526732005156104 - Best Solution 93.58974358974359 - Rolls No. 78.0
Iteration 1000 - Temperature 0.00011955860271661615 - Best Solution 93.58974358974359 - Rolls No. 78.

### The **best solution** after 2000 iteration is: 96 rolls


In [163]:
optimizer("input3.stock")

Iteration 100 - Temperature 0.40859382039498143 - Best Solution 105.50458715596329 - Rolls No. 109.00000000000001
Iteration 200 - Temperature 0.16544636987438172 - Best Solution 106.4814814814815 - Rolls No. 107.99999999999999
Iteration 300 - Temperature 0.06699196105841773 - Best Solution 109.52380952380953 - Rolls No. 104.99999999999999
Iteration 400 - Temperature 0.027126148792869235 - Best Solution 110.57692307692308 - Rolls No. 104.0
Iteration 500 - Temperature 0.010983824576970218 - Best Solution 114.99999999999999 - Rolls No. 100.00000000000003
Iteration 600 - Temperature 0.00444753154083448 - Best Solution 114.99999999999999 - Rolls No. 100.00000000000003
Iteration 700 - Temperature 0.0018008787984643694 - Best Solution 117.34693877551021 - Rolls No. 98.0
Iteration 800 - Temperature 0.0007292054967978848 - Best Solution 117.34693877551021 - Rolls No. 98.0
Iteration 900 - Temperature 0.00029526732005156104 - Best Solution 118.55670103092784 - Rolls No. 97.0
Iteration 1000 - Temp

### The **best solution** after 2000 iteration is: 217 rolls
Due to the fluctuations, the solution can explore a broader spectrum of possibilities, yielding results that outperform those obtained from the hill climbing algorithm. This particular input has exhibited more upward trends compared to the other three, resulting in a greater divergence in the output when compared with the hill climbing method.


In [162]:
optimizer("input4.stock")

Iteration 100 - Temperature 0.40859382039498143 - Best Solution 94.0 - Rolls No. 250.0
Iteration 200 - Temperature 0.16544636987438172 - Best Solution 97.91666666666666 - Rolls No. 240.00000000000003
Iteration 300 - Temperature 0.06699196105841773 - Best Solution 100.42735042735043 - Rolls No. 234.0
Iteration 400 - Temperature 0.027126148792869235 - Best Solution 103.52422907488987 - Rolls No. 227.0
Iteration 500 - Temperature 0.010983824576970218 - Best Solution 103.52422907488987 - Rolls No. 227.0
Iteration 600 - Temperature 0.00444753154083448 - Best Solution 105.85585585585586 - Rolls No. 221.99999999999997
Iteration 700 - Temperature 0.0018008787984643694 - Best Solution 105.38116591928251 - Rolls No. 223.0
Iteration 800 - Temperature 0.0007292054967978848 - Best Solution 106.81818181818181 - Rolls No. 220.00000000000003
Iteration 900 - Temperature 0.00029526732005156104 - Best Solution 106.81818181818181 - Rolls No. 220.00000000000003
Iteration 1000 - Temperature 0.00011955860271